# Eksempel, hvordan splitte vegobjekt i CSV 

Tusen takk for flott demo! 

In [5]:
import requests
import json
import pandas as pd

r = requests.get( 'https://apilesv3.utv.atlas.vegvesen.no/vegobjekter/67/460557991/8.json?&inkluder=vegsegmenter,lokasjon')
tun = r.json()

for seg in tun['vegsegmenter']:
    seg['kortform'] = seg['vegsystemreferanse']['kortform']

df = pd.DataFrame( tun['vegsegmenter'] ) 

In [20]:
df[['kortform', 'lengde', 'veglenkesekvensid', 'startposisjon', 'sluttposisjon']]

,kortform,lengde,veglenkesekvensid,startposisjon,sluttposisjon
0,RV706,14.938,2394247,0.000000,0.016029
1,RV706,916.968,2394247,0.016029,1.000000
2,RV706,1651.225,2394246,0.000000,0.967919
3,RV706,209.048,2394251,0.719539,1.000000
4,RV706,14.938,2394247,0.000000,0.016029
5,RV706,916.968,2394247,0.016029,1.000000
6,RV706,1651.225,2394246,0.000000,0.967919
7,RV706,209.048,2394251,0.719539,1.000000
8,RV706,14.938,2394247,0.000000,0.016029
9,RV706,916.968,2394247,0.016029,1.000000
